In [1]:
%load_ext autoreload
%autoreload 2

import torch

torch.set_grad_enabled(False)

In [2]:
from scripts.log_animation_usd import create_anim_from_usd

seq = create_anim_from_usd('/home/jorge/anim.usdc', n_frames=20)

100%|██████████| 20/20 [00:05<00:00,  3.77it/s]


In [17]:
from text3d2video.rendering import AnimSequence
from text3d2video.util import read_obj_with_uvs
from text3d2video.utilities.camera_placement import turntable_extrinsics
from pytorch3d.renderer import FoVPerspectiveCameras

mesh, verts_uvs, faces_uvs = read_obj_with_uvs('data/meshes/mixamo-human.obj')
R, T = turntable_extrinsics()
cams = FoVPerspectiveCameras(R=R, T=T, device='cuda', fov=30)
meshes = mesh.extend(len(R))

seq = AnimSequence(
    cams,
    mesh,
    verts_uvs,
    faces_uvs
)

  warnings.warn("No mtl file provided")



In [4]:
from text3d2video.artifacts.anim_artifact import AnimationArtifact

anim = AnimationArtifact.from_wandb_artifact_tag('joyful_jump_20:latest')
# frame_indices = anim.frame_indices(20)


frame_indices = [0, 7, 8, 9, 12]
seq = anim.read_anim_seq(frame_indices)

In [5]:
from text3d2video.rendering import render_rgb_uv_map
from text3d2video.utilities.video_comparison import display_vids
from text3d2video.utilities.video_util import pil_frames_to_clip

name = "joyful_jump_short"
uvs = render_rgb_uv_map(seq.meshes, seq.cams, seq.verts_uvs, seq.faces_uvs)
vid = pil_frames_to_clip(uvs, fps=15)
display_vids([vid], title=name)

In [6]:
from text3d2video.artifacts.anim_artifact import AnimationArtifact

new_anim = AnimationArtifact.create_empty_artifact(name)
new_anim.write_anim_seq(seq)
new_anim.log_standalone()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: romeu. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (/tmp/local_artifacts/animation/joyful_jump_short)... Done. 0.0s
